In [42]:
import open3d as o3d
from o3d_tools.visualize import PointCloudProject
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import torch
from torch_geometric.data import Data
import torch_geometric.transforms as T
#import torch_cluster

In [2]:
project1 = PointCloudProject(project='Project1') ; project2 = PointCloudProject(project='Project2')
project3 = PointCloudProject(project='Project3') ; project4 = PointCloudProject(project='Project4')

In [8]:
def read_and_preprocess_data():

    objs1 = project1.objects_df

    print(f"Types of objects in Project 1: {list(objs1.keys())}")
    pipes1 = objs1["Pipe"]
    scbs1 = objs1["Structural_ColumnBeam"]
    sibs1 = objs1["Structural_IBeam"]
    hvducts = objs1["HVAC_Duct"]

    objs2 = project2.objects_df

    print(f"Types of objects in Project 2: {list(objs2.keys())}")
    pipes2 = objs2["Pipe"]
    scbs2 = objs2["Structural_ColumnBeam"]
    sibs2 = objs2["Structural_IBeam"]

    objs3 = project3.objects_df

    print(f"Types of objects in Project 3: {list(objs3.keys())}")
    scbs3 = objs3["Structural_ColumnBeam"]

    objs4 = project4.objects_df

    print(f"Types of objects in Project 4: {list(objs4.keys())}")
    sibs4 = objs4["Structural_IBeam"]


    boxes1 = pd.concat([scbs1, hvducts, pipes1, sibs1])
    boxes2 = pd.concat([scbs2, pipes2, sibs2])
    boxes3 = scbs3
    boxes4 = sibs4

    boxes = [boxes1, boxes2, boxes3, boxes4]

    for i in range(len(boxes)):
        boxes[i].rename(columns={" Label": "label", " BB.Min.X " : "min_x", " BB.Min.Y " : "min_y", " BB.Min.Z " : "min_z",
                             " BB.Max.X " : "max_x", " BB.Max.Y " : "max_y", " BB.Max.Z" : "max_z"}, inplace=True)

    box_data = pd.concat(boxes)

    ids_to_points = {}

    projects = [project1, project2, project3, project4]

    for proj_id, box in enumerate(boxes):

        for i, ID in enumerate(box["ID"]):

            proj = projects[proj_id]

            # .pcd.crop() method expects this bounding box object, create it from the raw min max values
            bb = o3d.geometry.AxisAlignedBoundingBox((box[["min_x", "min_y", "min_z"]].iloc[i]).to_numpy(), 
                                        (box[["max_x", "max_y", "max_z"]].iloc[i]).to_numpy())

            points_of_id = proj.pcd.crop(bb)

            # convert the pointcloud object into a numpy array
            # join coordinates and color data into a single 6d nparray for each point

            points_coords = np.asarray(points_of_id.points)

            tmp = np.asarray(points_of_id.colors)[:,0]

            points_colors = (np.asarray(points_of_id.colors)[:,0]).reshape(-1,1)

            points_arr = np.concatenate((points_coords, points_colors), axis = 1)

            ids_to_points[ID] = points_arr

    def sample_point_cloud(points, num_points):

        if points.shape[0] > num_points:

            # Downsample if the point cloud has more points than needed
            idx = np.random.choice(points.shape[0], num_points, replace=False)
            return points[idx, :]

        elif points.shape[0] < num_points:
            # Upsample by repeating random points if the point cloud has fewer points than needed
            idx = np.random.choice(points.shape[0], num_points - points.shape[0], replace=True)
            return np.concatenate([points, points[idx, :]], axis=0)

        else:
            # Return the point cloud as is if it already has the correct number of points
            return points

    fixed_num_points = 2048  # The fixed number of points for all point clouds

    point_clouds_resampled = {}

    for obj_id in ids_to_points.keys():

        point_cloud = ids_to_points[obj_id]
        resampled_points = sample_point_cloud(point_cloud, fixed_num_points)
        point_clouds_resampled[obj_id] = resampled_points
        
    def extract_bounding_box_features(min_x, max_x, min_y, max_y, min_z, max_z):
    
        centroid = [(min_x + max_x) / 2, (min_y + max_y) / 2, (min_z + max_z) / 2]
        size = [max_x - min_x, max_y - min_y, max_z - min_z]
        volume = size[0] * size[1] * size[2]
        
        return np.array(centroid + size + [volume])
    
    data = []
    for i in range(box_data.shape[0]):

        box = box_data.iloc[i]
        features = extract_bounding_box_features(box["min_x"], box["max_x"], box["min_y"],
                                                 box["max_y"], box["min_z"], box["max_z"])

        data.append(features)

    feature_df = pd.DataFrame(data, columns=["cx", "cy", "cz", "sx", "sy", "sz", "vol"])
    
    box_data = box_data.reset_index(drop=True)
    
    new_box_data = pd.concat([box_data, feature_df], axis=1)
    
    return point_clouds_resampled, new_box_data

In [9]:
points_dict, box_data = read_and_preprocess_data()

Types of objects in Project 1: ['Structural_ColumnBeam', 'HVAC_Duct', 'Pipe', 'Structural_IBeam']
Types of objects in Project 2: ['Structural_ColumnBeam', 'Pipe', 'Structural_IBeam']
Types of objects in Project 3: ['Structural_ColumnBeam']
Types of objects in Project 4: ['Structural_IBeam']


In [22]:
y = box_data["label"]
X = box_data[["ID", "min_x", "min_y", "min_z", "max_x", "max_y", "max_z", "cx", "cy", "cz", "sx", "sy", "sz", "vol"]]

In [23]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

In [24]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(270, 14)
(90, 14)
(90, 14)
(270,)
(90,)
(90,)


In [35]:
def create_point_cloud_data(points, labels):
    
    # Points is of shape [2048, 3] (x, y, z coordinates)
    # Grayscale intensity is of shape [2048, 1] 
    # Labels are the classification targets for each bounding box

    x = torch.tensor(points, dtype=torch.float)  # Point cloud coordinates
    print(x)
    print(x.shape)

    # Create an edge index using k-nearest neighbors (or some method to define neighbors)
    edge_index = T.KNNGraph(k=16)(Data(pos=x)).edge_index  # Edge index for the graph

    # Create the Data object
    data = Data(x=x, edge_index=edge_index, y=torch.tensor(labels, dtype=torch.long))

    return data

In [37]:
!pip install torch-cluster

In [43]:
graph_dict = []

for obj_id in points_dict.keys():
    
    graph_dict[obj_id] = create_point_cloud_data(points_dict[obj_id], box_data[box_data["ID"] == obj_id])


tensor([[-2.2062e+02,  3.3525e+02,  8.5944e+01,  4.4141e-01],
        [-2.2178e+02,  3.3635e+02,  9.7627e+01,  1.5625e-02],
        [-2.2102e+02,  3.3530e+02,  9.5444e+01,  2.8125e-01],
        ...,
        [-2.2069e+02,  3.3543e+02,  9.3237e+01,  2.3047e-01],
        [-2.2155e+02,  3.3607e+02,  8.5803e+01,  3.4766e-01],
        [-2.1956e+02,  3.3600e+02,  8.5739e+01,  3.2422e-01]])
torch.Size([2048, 4])


ImportError: 'knn_graph' requires 'torch-cluster'